In [86]:
#Import dependants
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from pathlib import Path
import requests
import json
from pprint import pprint

from api_keys import geoapify_key

In [87]:
# load and read the CSVs
SHD_path = Path("Resources/SHD.csv")
SHD_csv = pd.read_csv(SHD_path)

SHD_csv.head()

,Incident_ID,Date,Quarter,school,City,State,School_Level,Location,During_School,Time_Period,First_Shot,Bullied,Domestic_Violence,Gang_Related,Preplanned
0,19700105DCHIW,1970-01-05,Winter,Hine Junior High School,Washington,DC,High,Inside School Building,Yes,NaN,NaN,No,No,No,No
1,19700105DCSOW,1970-01-05,Winter,Sousa Junior High,Washington,DC,Junior High,Hallway,Yes,NaN,NaN,No,No,No,No
2,19700105DCUNW,1970-01-05,Winter,Unnamed High School,Washington,DC,High,Playground,Yes,NaN,NaN,No,No,No,No
3,19700206OHJOC,1970-02-06,Winter,John F. Kennedy High School,Cleveland,OH,High,Hallway,Yes,NaN,NaN,No,No,No,No
4,19700323CADAL,1970-03-23,Spring,David Starr Jordan High School,Long Beach,CA,High,Beside Building,No,Night,NaN,No,No,No,No


In [88]:
# we have to make sure the column names match when it comes to lat and lon
# was working on getting the lat and lon for the Kaggle2_CSV when I realized we already have those
# we need lat and lon for the SHD.csv 
# capitalized the "S" in the school column for consistency
SHD_csv = SHD_csv.rename(columns={"school":"School"})
SHD_csv.head()

,Incident_ID,Date,Quarter,School,City,State,School_Level,Location,During_School,Time_Period,First_Shot,Bullied,Domestic_Violence,Gang_Related,Preplanned
0,19700105DCHIW,1970-01-05,Winter,Hine Junior High School,Washington,DC,High,Inside School Building,Yes,NaN,NaN,No,No,No,No
1,19700105DCSOW,1970-01-05,Winter,Sousa Junior High,Washington,DC,Junior High,Hallway,Yes,NaN,NaN,No,No,No,No
2,19700105DCUNW,1970-01-05,Winter,Unnamed High School,Washington,DC,High,Playground,Yes,NaN,NaN,No,No,No,No
3,19700206OHJOC,1970-02-06,Winter,John F. Kennedy High School,Cleveland,OH,High,Hallway,Yes,NaN,NaN,No,No,No,No
4,19700323CADAL,1970-03-23,Spring,David Starr Jordan High School,Long Beach,CA,High,Beside Building,No,Night,NaN,No,No,No,No


In [89]:
# Function to geocode a city and state and update the DataFrame
def geocode_and_update(city, state, api_key):
    query = f"{city}, {state}"
    url = f"https://api.geoapify.com/v1/geocode/search?text={query}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
    return None

# Example usage: Loop through cities and states in the DataFrame and geocode
api_key = geoapify_key
SHD_csv['lat'] = None  # Create new columns for latitude and longitude
SHD_csv['lon'] = None
for index, row in SHD_csv.iterrows():
    city = row['City']
    state = row['State']
    coordinates = geocode_and_update(city, state, api_key)
    if coordinates:
        SHD_csv.at[index, 'lat'] = coordinates[1]  # Assuming Geoapify returns latitude first
        SHD_csv.at[index, 'lon'] = coordinates[0]
        print(f"Geocoded {city}, {state}")
    else:
        print(f"Failed to geocode {city}, {state}")

# Save the updated DataFrame to a new CSV file
SHD_csv.to_csv('Resources/SHD_updated_locations.csv', index=False)
SHD_csv.head()


Geocoded Washington, DC
Geocoded Washington, DC
Geocoded Washington, DC
Geocoded Cleveland, OH
Geocoded Long Beach, CA
Geocoded Pine Bluff, AR
Geocoded Wilmington, DE
Geocoded Delray Beach, FL
Geocoded Ogden, UT
Geocoded El Paso, TX
Geocoded Miami, FL
Geocoded Nashville, TN
Geocoded Meridian , ID
Geocoded Davenport, IA
Geocoded Detroit, MI
Geocoded Memphis, TN
Geocoded Nashville, TN
Geocoded Mansfield, OH
Geocoded Chicago, IL
Geocoded Appleton, WI
Geocoded Philadelphia, PA
Geocoded Willingboro, PA
Geocoded Dallas, TX
Geocoded Cleveland, OH
Geocoded Tucson, AZ
Geocoded Harrisburg, PA
Geocoded Saint Joseph, MI
Geocoded East Aurora, NY
Geocoded Monessen, PA
Geocoded Lubbock, TX
Geocoded Buffalo, NY
Geocoded Minneapolis, MN
Geocoded Petersburg, VA
Geocoded Baltimore, MD
Geocoded Baltimore, MD
Geocoded Baltimore, MD
Geocoded Carlsbad, NM
Geocoded Tulsa, OK
Geocoded Baltimore, MD
Geocoded Los Angeles, CA
Geocoded Baltimore, MD
Geocoded Washington, DC
Geocoded Stow, OH
Geocoded Baltimore, MD


,Incident_ID,Date,Quarter,School,City,State,School_Level,Location,During_School,Time_Period,First_Shot,Bullied,Domestic_Violence,Gang_Related,Preplanned,lat,lon
0,19700105DCHIW,1970-01-05,Winter,Hine Junior High School,Washington,DC,High,Inside School Building,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
1,19700105DCSOW,1970-01-05,Winter,Sousa Junior High,Washington,DC,Junior High,Hallway,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
2,19700105DCUNW,1970-01-05,Winter,Unnamed High School,Washington,DC,High,Playground,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
3,19700206OHJOC,1970-02-06,Winter,John F. Kennedy High School,Cleveland,OH,High,Hallway,Yes,NaN,NaN,No,No,No,No,41.499657,-81.693677
4,19700323CADAL,1970-03-23,Spring,David Starr Jordan High School,Long Beach,CA,High,Beside Building,No,Night,NaN,No,No,No,No,33.769016,-118.191604


,Incident_ID,Date,Quarter,School,City,State,School_Level,Location,During_School,Time_Period,First_Shot,Bullied,Domestic_Violence,Gang_Related,Preplanned,lat,lon
0,19700105DCHIW,1970-01-05,Winter,Hine Junior High School,Washington,DC,High,Inside School Building,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
1,19700105DCSOW,1970-01-05,Winter,Sousa Junior High,Washington,DC,Junior High,Hallway,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
2,19700105DCUNW,1970-01-05,Winter,Unnamed High School,Washington,DC,High,Playground,Yes,NaN,NaN,No,No,No,No,4.652954,-74.083564
3,19700206OHJOC,1970-02-06,Winter,John F. Kennedy High School,Cleveland,OH,High,Hallway,Yes,NaN,NaN,No,No,No,No,41.499657,-81.693677
4,19700323CADAL,1970-03-23,Spring,David Starr Jordan High School,Long Beach,CA,High,Beside Building,No,Night,NaN,No,No,No,No,33.769016,-118.191604
